# CIFAR-10 classifier from nnBuilder

This is a simple implementation of a CIFAR-10 classifier using the nnBuilder framework. (Work in progress)

In [1]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append('../nnBuilder')
from nnLayer import *
from nnInput import *
from nnTrainer import *
from nnHandler import *
from _nnUtils import *

In [ ]:
data=Layer(x=None,type="CIFAR_10")
batch=Layer(x=data,type="Batch_Slice",batch=128)
layers=[]
layers.append({"type":"Convolution","pad":"SAME","window":5,"stride":1,"size":64,"relu":True})
layers.append({"type":"Pool","pad":"SAME","window":3,"stride":2,"pool_type":"max"})
layers.append({"type":"Convolution","pad":"SAME","window":5,"stride":1,"size":64,"relu":True})
layers.append({"type":"Pool","pad":"SAME","window":3,"stride":2,"pool_type":"max"})
layers.append({"type":"Relu","size":384})
layers.append({"type":"Relu","size":192})
layers.append({"type":"Linear","size":10,"dropout":True})
network=Layer(type="Network",x=batch,layers=layers)
trainer=ClassifierTrainer(network=network,optimizer="adam",finish=True)
sess=SessManager(data,batch,network,trainer)#,input_network)
sess.start()

Starting new session


In [ ]:
trainer.train(0,keep_rate=1.)
trainer.eval_error()
for i in range(20):
    trainer.train(1000,keep_rate=1.)
    trainer.train(0,keep_rate=1.)
    trainer.eval_error() #Training error

Error rate: 0.899219
